# CODE STILL IN PROGRESS, WILL MAKE SURE TO ADD ALL THE ELEMENTS ON THE RUBRIC FOR THE FINAL DRAFT

# Scientific Question

The protein Tau has several isoforms, how do they differ by their protein sequences using a BLAST search and pairwise sequence alignment?

# Scientific Hypothesis

If the isoforms are all known to be a factor for Alzheimer’s disease, then comparing the sequence of an isoform with another through a BLAST search may show a pattern on which isoforms are similar.

# LOADING PACKAGES

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from Bio import pairwise2
from Bio import SeqIO
from Bio.Seq import Seq 
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from itertools import combinations

# BLAST SEARCH

In [2]:
sequence_data = open("sequence.fasta").read()

In [3]:
sequence_data

'>NP_058519.3 microtubule-associated protein tau isoform 1 [Homo sapiens]\nMAEPRQEFEVMEDHAGTYGLGDRKDQGGYTMHQDQEGDTDAGLKESPLQTPTEDGSEEPGSETSDAKSTP\nTAEDVTAPLVDEGAPGKQAAAQPHTEIPEGTTAEEAGIGDTPSLEDEAAGHVTQEPESGKVVQEGFLREP\nGPPGLSHQLMSGMPGAPLLPEGPREATRQPSGTGPEDTEGGRHAPELLKHQLLGDLHQEGPPLKGAGGKE\nRPGSKEEVDEDRDVDESSPQDSPPSKASPAQDGRPPQTAAREATSIPGFPAEGAIPLPVDFLSKVSTEIP\nASEPDGPSVGRAKGQDAPLEFTFHVEITPNVQKEQAHSEEHLGRAAFPGAPGEGPEARGPSLGEDTKEAD\nLPEPSEKQPAAAPRGKPVSRVPQLKARMVSKSKDGTGSDDKKAKTSTRSSAKTLKNRPCLSPKHPTPGSS\nDPLIQPSSPAVCPEPPSSPKYVSSVTSRTGSSGAKEMKLKGADGKTKIATPRGAAPPGQKGQANATRIPA\nKTPPAPKTPPSSGEPPKSGDRSGYSSPGSPGTPGSRSRTPSLPTPPTREPKKVAVVRTPPKSPSSAKSRL\nQTAPVPMPDLKNVKSKIGSTENLKHQPGGGKVQIINKKLDLSNVQSKCGSKDNIKHVPGGGSVQIVYKPV\nDLSKVTSKCGSLGNIHHKPGGGQVEVKSEKLDFKDRVQSKIGSLDNITHVPGGGNKKIETHKLTFRENAK\nAKTDHGAEIVYKSPVVSGDTSPRHLSNVSSTGSIDMVDSPQLATLADEVSASLAKQGL\n\n'

In [65]:
result_handle = NCBIWWW.qblast("blastp", "refseq_select_prot", sequence_data,
                              filter = "Homo sapiens",
                              alignments = 1)
print(result_handle)

In [44]:
save_file = open("results.xml", "w")
save_file.write(result_handle.read())

14966

In [53]:
E_VALUE_THRESH = 1e-20
for record in NCBIXML.parse(open("results.xml")):
    if record.alignments:
        print("\n")
        print("query: %s" % record.query[:100])
        for align in record.alignments:
            for hsp in align.hsps:
                if hsp.expect < E_VALUE_THRESH:
                    print("match: %s" % align.title[:100])



query: NP_058519.3 microtubule-associated protein tau isoform 1 [Homo sapiens]
match: gi|1786986091|ref|NP_001364194.1| microtubule-associated protein tau isoform 9 [Homo sapiens]
match: gi|1786986091|ref|NP_001364194.1| microtubule-associated protein tau isoform 9 [Homo sapiens]
match: gi|84370347|ref|NP_001033698.1| microtubule-associated protein tau isoform a [Mus musculus]
match: gi|84370347|ref|NP_001033698.1| microtubule-associated protein tau isoform a [Mus musculus]
match: gi|890810631|ref|NP_001297563.1| microtubule-associated protein 2 isoform 3 [Mus musculus]
match: gi|1770726347|ref|NP_001362434.1| microtubule-associated protein 2 isoform 1 [Homo sapiens]
match: gi|328927075|ref|NP_001192259.1| microtubule-associated protein 4 isoform 1 [Mus musculus]
match: gi|1879298442|ref|NP_001372611.1| microtubule-associated protein 4 isoform 6 [Homo sapiens]


In [54]:
sequence = SeqIO.parse("sequence.fasta", "fasta")
print(type(sequence))
print(sequence)
for s in sequence:
    print(type(s))
    print(s)

<class 'Bio.SeqIO.FastaIO.FastaIterator'>
<class 'Bio.SeqRecord.SeqRecord'>
ID: NP_058519.3
Name: NP_058519.3
Description: NP_058519.3 microtubule-associated protein tau isoform 1 [Homo sapiens]
Number of features: 0
Seq('MAEPRQEFEVMEDHAGTYGLGDRKDQGGYTMHQDQEGDTDAGLKESPLQTPTED...QGL')


# PAIRWISE SEQUENCE ANALYSIS

In [57]:
def score(sequence1, sequence2):
    a = 0
    for pos in range(0, min(len(sequence1), len(sequence2))):
        if sequence1[pos] == sequence2[pos]:
            a += 2
        if sequence1[pos] != sequence2[pos]:
            a += -1
        if sequence1[pos] == sequence2[pos] == '-':
            a += -2
    return a

# this is comparing the two sequences using the score that was created above
def sequence_compare(file):
    seq_records = SeqIO.parse(file, "fasta")
    array = np.empty(1, int)
    for record1, record2 in combinations(seq_records, 2):
        sequence1 = record1.seq
        sequence2 = record2.seq
        a = score(sequence1, sequence2)
        array = np.append(array, [a], axis = 0)
        print(sequence1)
        print(sequence2)
        print(a)

sequence_compare("sequence.fasta")

# DOTPLOT

In [59]:
Fasta = list(SeqIO.parse("sequence.fasta", "fasta"))
isoform1 = Fasta[0].seq
print(isoform1)

MAEPRQEFEVMEDHAGTYGLGDRKDQGGYTMHQDQEGDTDAGLKESPLQTPTEDGSEEPGSETSDAKSTPTAEDVTAPLVDEGAPGKQAAAQPHTEIPEGTTAEEAGIGDTPSLEDEAAGHVTQEPESGKVVQEGFLREPGPPGLSHQLMSGMPGAPLLPEGPREATRQPSGTGPEDTEGGRHAPELLKHQLLGDLHQEGPPLKGAGGKERPGSKEEVDEDRDVDESSPQDSPPSKASPAQDGRPPQTAAREATSIPGFPAEGAIPLPVDFLSKVSTEIPASEPDGPSVGRAKGQDAPLEFTFHVEITPNVQKEQAHSEEHLGRAAFPGAPGEGPEARGPSLGEDTKEADLPEPSEKQPAAAPRGKPVSRVPQLKARMVSKSKDGTGSDDKKAKTSTRSSAKTLKNRPCLSPKHPTPGSSDPLIQPSSPAVCPEPPSSPKYVSSVTSRTGSSGAKEMKLKGADGKTKIATPRGAAPPGQKGQANATRIPAKTPPAPKTPPSSGEPPKSGDRSGYSSPGSPGTPGSRSRTPSLPTPPTREPKKVAVVRTPPKSPSSAKSRLQTAPVPMPDLKNVKSKIGSTENLKHQPGGGKVQIINKKLDLSNVQSKCGSKDNIKHVPGGGSVQIVYKPVDLSKVTSKCGSLGNIHHKPGGGQVEVKSEKLDFKDRVQSKIGSLDNITHVPGGGNKKIETHKLTFRENAKAKTDHGAEIVYKSPVVSGDTSPRHLSNVSSTGSIDMVDSPQLATLADEVSASLAKQGL


In [60]:
BLAST_hit = Fasta[1].seq
print(BLAST_hit)

IndexError: list index out of range

In [62]:
# Setting the window size and which sequence is on the x and y axes of the dot plot
window = 3
seq_one = str(isoform1)
seq_two = str(BLAST_hit)

# Saving the data in the dot plot based on window size and comparing for all I and all j in the range of the lengths of the
# sequences, wheter or not there is an alignment.

data = [
    [
        (seq_one[i : i + window] != seq_two[j: j + window])
        for j in range(len(seq_one) - window)
    ]
    for i in range(len(seq_two) - window)
]

NameError: name 'BLAST_hit' is not defined

In [63]:
import pylab

pylab.gray()
pylab.imshow(data)
pylab.xlabel("%s (length %i bp)" % (Fasta[0].id, len(Fasta[0].seq)))
pylab.ylabel("%s (length %i bp)" % (Fasta[1].id, len(Fasta[1].seq)))
pylab.title("Dot plot using window size %i\n(allowing no mis-matches)" % window)
pylab.show()

NameError: name 'data' is not defined

<Figure size 432x288 with 0 Axes>

# ANALYSIS

cannot analyze yet because I have errors